# Calcular score das avaliações

In [1]:
import numpy as np
import pandas as pd

In [2]:
# declare a list tasks whose products you want to use as inputs
upstream = None
product = None
df_input_path = '../../dados/X_trainToronto.csv'
df_review_input_path = '../../dados_produzidos/reviewsTrainTorontoSentiment.csv'

In [ ]:
df_reg = pd.read_csv(df_input_path)
df_reviews = pd.read_csv(df_review_input_path)
df_reg = df_reg[['business_id', 'review_count']]
df_reg

## Calcular score das analises

In [49]:
from sklearn.preprocessing import StandardScaler

def make_df_score(df, df_reviews):
    weights = {
        'useful': 1,
        'cool': 0.75,
        'funny': 0.5
    }
    df_reviews['weight'] = df_reviews.apply(lambda row: 1 + row['useful'] * weights['useful'] + row['cool'] * weights['cool'] + row['funny'] * weights['funny'], axis=1)
    def wavg_label(group):
        d = group['label']
        w = group['weight']
        return (d * w).sum() / w.sum()
    
    def wavg_score(group):
        d = group['score']
        w = group['weight']
        return (d * w).sum() / w.sum()
        
    df_reviews_label_grouped = df_reviews.groupby(['business_id']).apply(wavg_label, include_groups=False)
    df_reviews_score_grouped = df_reviews.groupby(['business_id']).apply(wavg_score, include_groups=False)
    df_reviews_scores = pd.DataFrame(df_reviews_label_grouped, columns=['stars']).join(pd.DataFrame(df_reviews_score_grouped, columns=['score']))
    df_score = df.join(df_reviews_scores)
    scaler = StandardScaler()
    df_score['stars'] = scaler.fit_transform(df_score[['stars']])
    return df_score

df_score = make_df_score(df_reg.set_index('business_id'), df_reviews)
df_reg[['stars', 'score']] = df_score.reset_index()[['stars', 'score']]
df_reg[['stars', 'score']]

,stars,score
0,0.582949,0.532271
1,0.791667,0.577041
2,0.340136,0.706820
3,0.666667,0.400056
4,0.006757,0.511156
...,...,...
17577,0.320359,0.567692
17578,0.772727,0.757226
17579,0.568000,0.657578
17580,0.146341,0.696177


In [ ]:
df_reg.to_parquet(product['data'])